In [1]:
import sys
sys.path.append('../tools')

In [2]:

from transformers import AutoModel
from train_utils import *
# from tools.train_utils import *

ModuleNotFoundError: No module named 'train_utils'

In [ ]:
PATH = Path('../data')
# PATH = Path('data')
BERT_PATH = '../data/bert-indo-15g'
# BERT_PATH = 'data/bert-indo-15g'
bert_model_file = '../data/saved_models/text_model/fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08.pth'
# bert_model_file = 'data/saved_models/text_model/fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08.pth'


In [ ]:
train_df = pd.read_csv(PATH/'train.csv')
dummy_df = train_df.iloc[:2].copy()
dummy_df['is_valid'] = False
train_df['is_valid'] = True
train_df= pd.concat([train_df, dummy_df])

In [ ]:
dummy_df

,posting_id,image,image_phash,title,label_group,is_valid
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,False
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,False


In [ ]:
class CONF( ConfigClass):
    bs = 256
    embedding_size = 768

conf = CONF()

In [ ]:
class BertArcFace(nn.Module):
    def __init__(self, embedding_size=768):
        super().__init__()
        self.bert_model = AutoModel.from_pretrained(BERT_PATH)
        self.collected = torch.empty((0,embedding_size)).cuda()
    def forward(self, x):
        output = self.bert_model(*x)
        return output.last_hidden_state[:,0,:]

In [ ]:
def load_bert_model(fname):
    model = BertArcFace(conf.embedding_size)
    state = torch.load(fname)
    model.load_state_dict(state)
    return model

In [ ]:
model = load_bert_model(bert_model_file)


Some weights of the model checkpoint at ../data/bert-indo-15g were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
learn = Learner(get_text_dls(df,BERT_PATH, bs), model=model)
embeddings, y = learn.get_preds()

In [ ]:
# def get_embeddings(model,df,BERT_PATH,bs=64):
#     learn = Learner(get_text_dls(df,BERT_PATH, bs), model=model, loss_func=NoLoss(),cbs=GetEmbs())
#     learn.validate()
#     return learn.model.collected
 

In [ ]:
# %debug

In [ ]:
# embeddings = get_embeddings(model, train_df, BERT_PATH, conf.bs).cpu()

KeyboardInterrupt: 

In [ ]:
# save_folder =  '../data/saved_embeddings/text_embeddings/'
# save_name = 'fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08_embed.pth'

In [ ]:
# torch.save(embeddings, save_folder + save_name)